In [ ]:
from models import getModel 
from dataloader import data_pipeline, data_pipeline_with_augmentation
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B3
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.regnet import RegNetX032 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, CSVLogger, EarlyStopping 
import pandas as pd
import pickle 


In [ ]:
preTrained = [
    ('EfficientNetB5', EfficientNetB5),
    ('EfficientNetV2B3', EfficientNetV2B3),
    ('ResNet50V2', ResNet50V2),
    ('InceptionResNetV2', InceptionResNetV2),
    ('VGG16', VGG16),
    ('MobileNetV2', MobileNetV2),
    ('RegNetX032', RegNetX032)
]

In [ ]:
def train():
    for preTrainedModel in preTrained:
        trainDataset, validateDataset = data_pipeline() 
        model = getModel(preTrainedModel[1])

        callbacks = [ 
            EarlyStopping(patience=10, verbose=1), 
            ReduceLROnPlateau(factor=0.5, cooldown=0, patience=3, min_lr=0.5e-6), 
            CSVLogger(f'{preTrainedModel[0]}_no_augmentation.csv'), 
            ModelCheckpoint(f'{preTrainedModel[0]}_no_augmentation.hdf5', monitor = 'val_accuracy',verbose = 1, save_best_only=True)
        ]

        model.fit(trainDataset, batch_size=4, epochs=30, verbose=1,validation_data=validateDataset, shuffle=False, callbacks=callbacks)
        model.load_weights(f'{preTrainedModel[0]}_no_augmentation.hdf5')

        results = model.get_metrics_result()
        df = pd.DataFrame(results).transpose()

        pickle.dump(df.to_latex(), open(f'{preTrainedModel[0]}_classification_report_acc={round(results)}.txt', 'wb'))
